In [1]:
import pandas as pd
import os
import datetime
import pickle
import re
import time
from collections import Counter
import numpy as np

import nltk
nltk.data.path

['/home/ec2-user/nltk_data',
 '/home/ec2-user/SageMaker/.conda/envs/my_py/nltk_data',
 '/home/ec2-user/SageMaker/.conda/envs/my_py/share/nltk_data',
 '/home/ec2-user/SageMaker/.conda/envs/my_py/lib/nltk_data',
 '/usr/share/nltk_data',
 '/usr/local/share/nltk_data',
 '/usr/lib/nltk_data',
 '/usr/local/lib/nltk_data']

In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize
#nltk.download('stopwords')
from nltk.corpus import stopwords

In [3]:
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [4]:
import random

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
# Define a text preprocessor (lemmatizer)
def my_preprocessor(text):
    doc=nlp(text.lower())
    lemmas=[token.lemma_ for token in doc if not token.is_punct | token.is_space]
    texts_out=" ".join(lemmas)
    return texts_out

In [7]:
# Use the dictionary of regulatory noun chunks
df_nounchunks=pd.read_csv('/home/ec2-user/SageMaker/New Uncertainty/DictionaryOfRegulatoryNounChunks.csv')
print(df_nounchunks.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10684 entries, 0 to 10683
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   nc_code      10684 non-null  int64  
 1   noun_chunks  10684 non-null  object 
 2   rin          10684 non-null  object 
 3   area         10684 non-null  object 
 4   area_no      10684 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 417.5+ KB
None


In [8]:
nounchunks=df_nounchunks['noun_chunks'].tolist()
print(len(nounchunks),nounchunks[0:20])

10684 ['1 capital', '10 guideline', '10 specie', '10 usc', '103 regulation', '11 plant', '11 usc', '12 cfr', '12 specie', '13 plant', '15 cfr', '16 compound', '16 plant', '18 plant', '18 u.s.c', '180-day exclusivity', '1983 amendment', '1988 amendment', '1990 farm bill', '1993 provision']


In [9]:
# Import expanded reg sentences
df_regSentsExpand=pd.read_pickle('/home/ec2-user/SageMaker/New Uncertainty/Update Data/allRegSentsExpand_Update.pkl')
print(df_regSentsExpand.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9278 entries, 0 to 9277
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID                9278 non-null   object        
 1   Title             9278 non-null   object        
 2   Type              9278 non-null   object        
 3   StartDate         9278 non-null   datetime64[ns]
 4   EndDate           9278 non-null   object        
 5   Text              9278 non-null   object        
 6   TextWordCount     9278 non-null   object        
 7   PubTitle          9278 non-null   object        
 8   SourceType        9278 non-null   object        
 9   Year              9278 non-null   int64         
 10  Month             9278 non-null   int64         
 11  Newspaper         9278 non-null   object        
 12  RegSentsExpand    9278 non-null   object        
 13  RegSentExpandLen  9278 non-null   int64         
dtypes: datetime64[ns](1), in

In [10]:
# Convert reg sentences to list
regSentsExpand=df_regSentsExpand['RegSentsExpand'].tolist()
print(len(regSentsExpand), regSentsExpand[0])

9278 Matsuyama hit just 22 of 56 fairways and just 37 of 72 greens in regulation and still managed a 2-under total and a tie for third. ■ I wouldn't mind having Hideki Matsuyama's short game for a weekend. He got up and down from everywhere and was tops in strokes gained around the green.


In [11]:
print(regSentsExpand[4031])
print(my_preprocessor(regSentsExpand[4031]))

The low attendance is caused by several factors. Udvar-Hazy and the zoo are open daily, but the other sites are on five-day schedules. Government coronavirus regulations require museums to limit attendance to a small percentage of capacity, and museum officials have chosen to limit hours of operation to allow for cleaning.
the low attendance be cause by several factor udvar hazy and the zoo be open daily but the other site be on five day schedule government coronavirus regulation require museum to limit attendance to a small percentage of capacity and museum official have choose to limit hour of operation to allow for clean


In [12]:
# Preprocess all expanded reg sentences
regSentsExpand_lemmatized=[my_preprocessor(sent) for sent in regSentsExpand]

In [13]:
print(len(regSentsExpand_lemmatized), regSentsExpand_lemmatized[0])

9278 matsuyama hit just 22 of 56 fairway and just 37 of 72 green in regulation and still manage a 2-under total and a tie for third ■ i would not mind have hideki matsuyama 's short game for a weekend -PRON- get up and down from everywhere and be top in stroke gain around the green


In [14]:
print(regSentsExpand[0],regSentsExpand_lemmatized[0])

Matsuyama hit just 22 of 56 fairways and just 37 of 72 greens in regulation and still managed a 2-under total and a tie for third. ■ I wouldn't mind having Hideki Matsuyama's short game for a weekend. He got up and down from everywhere and was tops in strokes gained around the green. matsuyama hit just 22 of 56 fairway and just 37 of 72 green in regulation and still manage a 2-under total and a tie for third ■ i would not mind have hideki matsuyama 's short game for a weekend -PRON- get up and down from everywhere and be top in stroke gain around the green


In [15]:
# Compile a new re pattern with regulatory noun chunks
pattern=re.compile(r"\b"+r"\b|\b".join(map(re.escape, nounchunks))+r"\b")

In [16]:
# Match noun chunks in all expanded reg sentences
start_time = time.time()

nounchunk_match=[]
nounchunk_match_words=[]
for sent in regSentsExpand_lemmatized:
    match_words=[]
    match=0
    find=pattern.findall(sent)
    if len(find)>0:
        match_words=find
        match=len(find)
    nounchunk_match.append(match)
    nounchunk_match_words.append(match_words)
    
print("--- %s seconds ---" % (time.time() - start_time))

--- 252.3126175403595 seconds ---


In [17]:
print(len(nounchunk_match), len(nounchunk_match_words))
print(nounchunk_match_words[-1], nounchunk_match[-1])

9278 9278
['u.s government'] 1


In [18]:
# Export results: matched noun chunks in expanded reg sentences
df_regSentsExpand['NounChunkMatchFiltered']=nounchunk_match
df_regSentsExpand['NounChunkMatchWordsFiltered']=nounchunk_match_words

In [19]:
print(df_regSentsExpand.head())

           ID                                              Title  Type  \
0  2438846341                1-vs.-2 duel ended much too quickly  News   
1  2438796738  Bruins eliminated in a double-overtime thrille...  News   
2  2439088192  Crowded field vies for the Fourth District pri...  News   
3  2438846397      Furious final hours for an unparalled primary  News   
4  2439087001  In battle for the future, Markey and Kennedy r...  News   

   StartDate     EndDate                                               Text  \
0 2020-09-01  2020-09-01  I was rooting for Jon Rahm and Dustin Johnson ...   
1 2020-09-01  2020-09-01  This was the longest and likely, the strangest...   
2 2020-09-01  2020-09-01  Voters from Brookline to Fall River cast ballo...   
3 2020-09-01  2020-09-01  On the eve of an unprecedented primary,empty b...   
4 2020-09-01  2020-09-01  For the first time in Massachusetts political ...   

  TextWordCount               PubTitle  SourceType  Year  Month     Newspaper  \

In [20]:
print(df_regSentsExpand[df_regSentsExpand['NounChunkMatchFiltered']>0].info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5832 entries, 5 to 9277
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID                           5832 non-null   object        
 1   Title                        5832 non-null   object        
 2   Type                         5832 non-null   object        
 3   StartDate                    5832 non-null   datetime64[ns]
 4   EndDate                      5832 non-null   object        
 5   Text                         5832 non-null   object        
 6   TextWordCount                5832 non-null   object        
 7   PubTitle                     5832 non-null   object        
 8   SourceType                   5832 non-null   object        
 9   Year                         5832 non-null   int64         
 10  Month                        5832 non-null   int64         
 11  Newspaper                    5832 non-null 

In [21]:
for i in range(0,100):
    if df_regSentsExpand['NounChunkMatchFiltered'][i]>0:
        print(df_regSentsExpand['RegSentsExpand'][i],df_regSentsExpand['NounChunkMatchWordsFiltered'][i])

The National Oceanic and Atmospheric Administration said it established a “dynamic management area” south of Nantucket where “an aggregation of right whales” was seen on Monday. Federal fisheries regulators are asking mariners to either go slow or find a route around an area south of Nantucket where groups of right whales have recently been spotted as the endangered mammals migrate. ['national oceanic', 'atmospheric administration', 'management area', 'right whale', 'federal fishery', 'right whale']
The 50,000 home health aides who work for independent agencies, on the other hand, have less oversight, lower pay, and less political power to demand protective equipment and hazard pay, she said. Four Black workers grace 13 billboards bearing the messages: “Black homecare workers are leading the fight against COVID — and racism” and “Homecare workers are fighting to be seen and heard because our lives matter. ”The goal is to prod elected officials in Massachusetts to invest in home care an

In [22]:
df_regSentsExpand.to_pickle('/home/ec2-user/SageMaker/New Uncertainty/Update Data/RegSentsExpand_NounChunks_Update.pkl')